In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [7]:
diabetes = pd.read_csv('data/diabetes.csv')

In [8]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [9]:
drop_indices = np.random.choice(diabetes.index, 368, replace=False)
diabetes = diabetes.drop(drop_indices)

In [10]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'], test_size=0.2)

In [11]:
X_train[X_train == 0] = 0.000001
X_test[X_test == 0] = 0.000001

In [12]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [13]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

In [14]:
"""
with open('results/version4/train_data.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version4/test_data.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

In [2]:
with open('results/version4/train_data.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version4/train_data.json') as json_file:
    y_train = json.load(json_file)[1]
    
with open('results/version4/test_data.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version4/test_data.json') as json_file:
    y_test = json.load(json_file)[1]

## Test

In [3]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=20, 
                           trainingBias=False,
                           epoch_results=True,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs','phase-encoding'})

lrParameter:  0.1
threshold:  0.7
trainingBias:  False
phaseEstrategyOperator:  original

erro HSGS 378
weights HSGS [7.81764705882351, 21.26231155778894, 21.545901639344233, 7.807070707070713, 3.5065011820330954, 17.632488822652768, 5.9855678906917165, 6.783333333333335]
erro phase encoding 199
weights phase encoding [-5.088235294117638, -13.376381909547746, -10.847540983606555, -4.022222222222223, -1.6855791962174944, -9.999105812220566, -3.5094790777113576, -4.6000000000000005]

erro HSGS 378
weights HSGS [15.135294117647014, 42.02462311557788, 42.591803278688495, 15.114141414141413, 6.513002364066194, 34.764977645305514, 11.47113578138343, 13.066666666666682]
erro phase encoding 198
weights phase encoding [-10.682352941176442, -27.288442211055294, -22.234426229508177, -8.562626262626267, -3.8801418439716318, -20.52861400894189, -7.529419299743811, -9.701666666666672]

erro HSGS 378
weights HSGS [37.08823529411854, 104.31155778894531, 105.72950819672096, 37.035353535353615, 15.53250

In [4]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=10,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs', 'phase-encoding'})

AVG TEST ERROR HSGS    0.6354
AVG TEST ERROR PHASE   0.3645999999999999


In [5]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

#print('phase neuron_outputs: ', results[1])
#print('HSGS neuron_outputs: ', results[4])
#print('targets: ', results[6])
#print('examples: ', results[7])

phase avg_error 0.3645999999999999
phase weights [-111.08235294117804, -276.87035175879487, -226.45573770491774, -90.00101010101095, -43.24219858156013, -209.41639344262342, -79.63181041844564, -101.32833333333211]
\HSGS avg_error 0.6354
HSGS weights [146.85294117647211, 415.7462311557764, 421.4180327868815, 146.64141414141466, 60.63002364066266, 343.14977645305936, 110.21135781383398, 126.16666666666406]


## Search

In [6]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [7]:
# search space

initial_weights = [
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
 #[0.9091283748867313, 0.13316944575925016],
 #[0.5234125806737658, 0.7504098591020348],
 #[0.6690132408839138, 0.4677528597449807],
 #[0.20484909029779508, 0.49076588909107044],
 #[0.372384689385059, 0.4774011548515884]
]

threshold_space = [ 0.8, 0.2, 0.5]
lr_space = [0.01, 0.05, 0.1]
bias_space = [False, True]
strategy_space = [ 'angleradius', 'radius', 'original', 'angle']

In [8]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=w,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs','phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=5,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'hsgs', 'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)

                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')


lrParameter:  0.01
threshold:  0.8
trainingBias:  False
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  123
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.325


lrParameter:  0.01
threshold:  0.8
trainingBias:  True
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  124
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.325


lrParameter:  0.05
threshold:  0.8
trainingBias:  False
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  124
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.325


lrParameter:  0.05
threshold:  0.8
trainingBias:  True
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  124
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.325


lrParameter:  0.1
threshold:  0.8
trainingBias:  False
phaseEstrategyOperator:  angleradius

best 

KeyboardInterrupt: 

In [9]:
experiment_data = pd.DataFrame(output)
experiment_data

,model,phase_strategy,bias,threshold,lr,avg_error,trained_weights,initial_weights,neuron_outputs
0,phase-encoding,angleradius,False,0.8,0.01,0.325,"[-19.489089277599792, -15.382001818012048, -13...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,HSGS,angleradius,False,0.8,0.01,0.675,"[8.217653858823546, 22.312060701507576, 22.560...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,phase-encoding,angleradius,True,0.8,0.01,0.325,"[-19.49814646840606, -15.38916589660257, -13.3...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,HSGS,angleradius,True,0.8,0.01,0.675,"[8.217653858823546, 22.312060701507576, 22.560...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,phase-encoding,angleradius,False,0.8,0.05,0.325,"[-99.49073234202976, -78.94582948301232, -68.8...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,HSGS,angleradius,False,0.8,0.05,0.675,"[39.088269294118824, 109.56030350753747, 110.8...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,phase-encoding,angleradius,True,0.8,0.05,0.325,"[-99.49073234202976, -78.94582948301232, -68.8...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,HSGS,angleradius,True,0.8,0.05,0.675,"[39.088269294118824, 109.56030350753747, 110.8...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,phase-encoding,angleradius,False,0.8,0.10,0.325,"[-199.31419576598444, -158.27854421192606, -13...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,HSGS,angleradius,False,0.8,0.10,0.675,"[77.67653858823769, 218.6206070150749, 221.106...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [10]:
experiment_data.to_csv('results/experiment_diabetes_angleradius.csv')